In [1]:
from fractions import Fraction as F

In [2]:
f_CLK = F(40000000)  # 40 MHz crystal
scaler = [F(1), F(1, 2), F(1, 4), F(2)]

## BPSK beacon

Sample rate 1 Msps

LO frequency 2400062500 kHz

```
cd /sys/bus/iio/devices/iio\:device1/
```

```
# cat tx_path_rates
BBPLL:1024000000 DAC:32000000 T2:16000000 T1:8000000 TF:4000000 TXSAMP:1000000
```

BBPLL registers:
```
# iio_reg ad9361-phy 0x41
0x13
# iio_reg ad9361-phy 0x42
0x20
# iio_reg ad9361-phy 0x43
0x0
# iio_reg ad9361-phy 0x44
0x19
# iio_reg ad9361-phy 0x45
0x0
```

In [3]:
bbpll_N_int = 0x19  # reg 0x44
bbpll_N_frac = 0x13 << 16 | 0x20 << 8 | 0x0  # reg 0x41 << 16 | reg 0x42 << 8 | reg 0x43
bbpll_ref_scaler = scaler[0x0 & 0x3] # reg 0x45 & 0x3
bbpll_N_int, bbpll_N_frac, bbpll_ref_scaler

(25, 1253376, Fraction(1, 1))

In [4]:
bbpll_f = f_CLK * bbpll_ref_scaler * (F(bbpll_N_int) + F(bbpll_N_frac) / F(2088960))
bbpll_f

Fraction(1024000000, 1)

In [5]:
samp_rate = bbpll_f / F(1024)
samp_rate

Fraction(1000000, 1)

TX synthesizer registers:

```
# iio_reg ad9361-phy 0x271
0x78
# iio_reg ad9361-phy 0x272
0x0
# iio_reg ad9361-phy 0x273
0x66
# iio_reg ad9361-phy 0x274
0x66
# iio_reg ad9361-phy 0x274
0x0
# iio_reg ad9361-phy 0x275
0x0
```

TX RFPLL reference divider:

```
# iio_reg ad9361-phy 0x2ac
0xff
```

RFPLL output dividers (chip level setup registers):

```
# iio_reg ad9361-phy 0x5
0x13
```

In [6]:
txpll_N_int = (0x0 & 0x7) << 8 | 0x78  # (reg 0x272 & 0x7) << 8 | reg 0x271
txpll_N_frac =  (0x0 & 0x7f) << 16 | 0x66 << 8 | 0x66  # (reg 0x275 & 0x7f) << 16 | reg 0x274 << 8 | reg 0x273
txpll_div = 1 << ((0x13 >> 4) + 1)  # 1 << ((reg 0x05 >> 4) + 1)
txpll_ref_scaler = scaler[(0xff >> 2) & 0x3]  # (reg 0x2ac >> 2) & 0x3

txpll_N_int, txpll_N_frac, txpll_div, txpll_ref_scaler

(120, 26214, 4, Fraction(2, 1))

In [7]:
txpll_f = f_CLK * txpll_ref_scaler * (F(txpll_N_int) + F(txpll_N_frac) / F(8388593))
txlo_f = txpll_f / F(txpll_div)
float(txlo_f)

2400062499.158083

In [8]:
error_bpsk = txlo_f - F(2400062500)
error_bpsk, float(error_bpsk)

(Fraction(-7062500, 8388593), -0.8419171129175059)

## 8APSK beacon

Sample rate 1.8 Msps (not important, since the beacons is at DC in IQ baseband)
LO frequency 2400495000 kHz

```
# iio_reg ad9361-phy 0x271
0x78
# iio_reg ad9361-phy 0x272
0x0
# iio_reg ad9361-phy 0x273
0x2
# iio_reg ad9361-phy 0x274
0x2b
# iio_reg ad9361-phy 0x275
0x3
# iio_reg ad9361-phy 0x2ac
0xff
# iio_reg ad9361-phy 0x5
0x13
```

In [9]:
txpll_N_int = (0x0 & 0x7) << 8 | 0x78  # (reg 0x272 & 0x7) << 8 | reg 0x271
txpll_N_frac =  (0x3 & 0x7f) << 16 | 0x2b << 8 | 0x2  # (reg 0x275 & 0x7f) << 16 | reg 0x274 << 8 | reg 0x273
txpll_div = 1 << ((0x13 >> 4) + 1)  # 1 << ((reg 5 >> 4) + 1)
txpll_ref_scaler = scaler[(0xff >> 2) & 0x3]  # (reg 0x2ac >> 2) & 0x3

txpll_N_int, txpll_N_frac, txpll_div, txpll_ref_scaler

(120, 207618, 4, Fraction(2, 1))

In [10]:
txpll_f = f_CLK * txpll_ref_scaler * (F(txpll_N_int) + F(txpll_N_frac) / F(8388593))
txlo_f = txpll_f / F(txpll_div)
float(txlo_f)

2400495000.7706895

In [11]:
error_8apsk = txlo_f - F(2400495000)
error_8apsk, float(error_8apsk)

(Fraction(6465000, 8388593), 0.770689435045901)

In [12]:
diff_error = error_8apsk - error_bpsk
diff_error, float(diff_error)

(Fraction(13527500, 8388593), 1.612606547963407)